# Basic Example for Image Classification


In [ ]:
import os
import shutil
import tarfile
import tempfile
import tensorflow.compat.v1 as tf
print(tf.__version__)

In [ ]:
import tensorflow_hub as hub; 
print("Hub version:", hub.__version__)

In [ ]:
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

## Working with flower image set
Downloading and extracting image set:

In [ ]:
print('Beginning tf models download with requests')
os.makedirs('build', exist_ok=True) 
tf.keras.utils.get_file(os.path.realpath("build/flower_photos"), 
                    "http://download.tensorflow.org/example_images/flower_photos.tgz",
                    untar=True, extract=True, cache_dir=os.path.join(os.getcwd()+"/build"), cache_subdir="") 
print('File unzipped')

## Visualization (optional)
It is possible to visualize some details of a training using TensorBoard. You can run TensorBoard in advance to have a continuous visualization, or run it after everything is done to have an overview. TensorBoard reads and visualizes the logs of a training. Thus, if it is started in advance, then it might show a "No dashboard" message, or a previous training first, but it should load some graphs shortly after the training starts.

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir build/logs

In [ ]:
# Some environments cannot clean up Tensorboard temp files, which leads to an error message when the tensorboard is started, 
# saying that you have to kill the process. In such cases uncomment the following lines, or delete the .temp files manually, 
# and try running tensorboard again.

#tempdir = os.path.join(tempfile.gettempdir(), ".tensorboard-info")
#if os.path.exists(tempdir):
#    shutil.rmtree(tempdir)

### Re-train with flower images

In [ ]:
%run scripts/retrain.py --image_dir build/flower_photos --tfhub_module https://tfhub.dev/google/imagenet/mobilenet_v1_100_224/feature_vector/3 --output_labels build/flower_labels.txt --output_graph build/flowers_graph.pb --how_many_training_steps 25 --summaries_dir build/logs

### Validating with flowers
Take a random picture from the flower image set to see if the network classifies it correctly:

In [ ]:
%run scripts/label_image.py --image build/flower_photos/roses/7186509956_c37c02fb43_n.jpg --graph build/flowers_graph.pb --labels build/flower_labels.txt --input_layer module_apply_default/hub_input/Mul --output_layer final_result

## Working with SIMATIC image set
Extracting image set:

In [ ]:
with tarfile.open("../../datasets/simatic_photos.tgz") as tar:
    tar.extractall('build/')
print('File unzipped')

### Re-train network with SIMATIC images

In [ ]:
%run scripts/retrain.py --image_dir build/simatic_photos --tfhub_module https://tfhub.dev/google/imagenet/mobilenet_v1_100_224/feature_vector/3 --output_labels build/retrained_simatic_labels.txt --output_graph build/simatic_graph.pb --how_many_training_steps 25 --summaries_dir build/logs

### Validating with SIMATIC images

Now having the script, take a random picture from the SIMATIC image set to see if the network classifies it correctly:

In [ ]:
%run scripts/label_image.py --image build/simatic_photos/ET200sp/IMG_0371.JPG --graph build/simatic_graph.pb --labels build/retrained_simatic_labels.txt --input_layer module_apply_default/hub_input/Mul --output_layer final_result